In [1]:
import pandas as pd

In [2]:
# read csvs as data frames
# cast as string to avoid float/NaN errors

# scenes = pd.read_csv('in/scenes.csv', dtype=object)
# scenes_productions = pd.read_csv('in/scenes_productions.csv', dtype=object)
# shamisenplayers = pd.read_csv('in/sceneshamisens.csv', dtype=object)
# musicians = pd.read_csv('in/scenekotokokyus.csv', dtype=object)
# narrators = pd.read_csv('in/scenetayus.csv', dtype=object)
# spucks = pd.read_csv('in/spucks.csv', dtype=object)
# scenes_images = pd.read_csv('in/scenes_images.csv', dtype=object)
# imagesonline = pd.read_csv('in/imagesonline.csv', dtype=object)

In [3]:
scenes = scenes[['id','label_eng','label_ja','label_ka','play_sequence']]
# scenes = scenes.rename(columns = {'play_sequence':'scene_order', 'id':'scene_id'})
# scenes.head()

,scene_id,label_eng,label_ja,label_ka,scene_order
0,1,Scene One,(1),（一）,001
1,2,Scene Two,(2),（二）,002
2,3,Scene Three,(3),（三）,003
3,4,The Yamanaya Tea House,Yamanaya,山名屋,004
4,5,Scene One,(1),（一）,005


In [4]:
# # clean out non-online images
# imagesonline = imagesonline[['image_id']]
# scenes_images = pd.merge(scenes_images, imagesonline, on='image_id', how='inner')

In [5]:
pscenes = pd.merge(scenes_productions, scenes, on='scene_id', how='left')

In [6]:
pscenes = pscenes.drop('spuck_note',1).drop('tayu_shamisen_note',1)
pscenes = pscenes.rename(columns = {'id':'pscene_id'})
pscenes.head()

,pscene_id,play_id,prod_id,perf_id,scene_id,label_eng,label_ja,label_ka,scene_order
0,1,86,1,2,313,The Tea Stall at Torii Pass,Torii tōge chamise,鳥居峠茶店,408
1,2,86,1,2,612,Ao no Dōmon (Blue Tunnel),Ao no Dōmon,青の洞門,409
2,3,29,1,3,163,The Town of Numazu,Numazu,沼津,189
3,4,29,1,3,641,House of Heisaku and One Thousand Pine Trees a...,Heisaku uchi yori senbon matsubara,平作内より千本松原,196
4,5,47,1,4,287,The Maple Viewing Party,Momijigari,紅葉狩,360


In [7]:
spucks = spucks.rename(columns = {'sceneproduction_id':'pscene_id','kashira1_id':'kashira_id','sortorder':'sort'})
spucks = spucks[['id','pscene_id','puppeteer_id','character_id','kashira_id','sort']]
spucks.head()

,id,pscene_id,puppeteer_id,character_id,kashira_id,sort
0,1,14,108,113,36,1
1,2,14,100,114,116,2
2,3,14,130,115,33,3
3,4,14,135,116,94,4
4,5,14,136,116,94,5


In [8]:
# export spucks
# spucks.to_csv('spucks_exp.csv', encoding='utf8', index=False)
# spucks.to_json('spucks_exp.json', orient='records', force_ascii=False)

In [1]:
# make a spuck / pscene join table
spuck_pscene = spucks.rename(columns = {'id':'spuck_id'})
spuck_pscene = spuck_pscene[['spuck_id','pscene_id']]
spuck_pscene.head()

NameError: name 'spucks' is not defined

In [10]:
# multiValDataFrame takes a dataframe df, column name df_index and column name multival_key
# it merges records with the same df_index and updates multival_index to an array of all the multival_index values.
# E.g for a data frame plays with multiple character_id values per play_id, returns a data frame with one record 
# per play_id, and one array of character_ids per play.

def multiValDataFrame(df, df_index, multival_key):
    new_df = pd.DataFrame({})
    cols = df.columns
    id_list = df[df_index].unique()
    for i in id_list:
        temp_row = df.loc[df[df_index] == i ]
        vals = []
        for col in cols:
            vals.append(temp_row[:1][col].values[0])

        temp_dict = dict(zip(cols, vals))
        multi_id_list = list(temp_row[multival_key])
    
        temp_dict[multival_key] = multi_id_list
    
        new_df = new_df.append(temp_dict, ignore_index=True)
    return new_df

In [11]:
pscenes = pd.merge(pscenes, spuck_pscene, on='pscene_id', how='left')
pscenes = multiValDataFrame(pscenes, 'pscene_id', 'spuck_id')

In [12]:
scenes_images = scenes_images.rename(columns = {'scene_id':'pscene_id'})
pscenes = pd.merge(pscenes, scenes_images, on='pscene_id', how='left')
pscenes = multiValDataFrame(pscenes, 'pscene_id', 'image_id')

In [13]:
# turn shamisenplayer df into join table
shamisenplayers = shamisenplayers.rename(columns = {'sceneproduction_id':'pscene_id'})
shamisen_pscene = shamisenplayers[['shamisen_id','pscene_id']]
shamisen_pscene.head()

,shamisen_id,pscene_id
0,104,14
1,67,14
2,65,15
3,110,15
4,40,16


In [14]:
pscenes = pd.merge(pscenes, shamisen_pscene, on='pscene_id', how='left')

In [15]:
pscenes = multiValDataFrame(pscenes, 'pscene_id', 'shamisen_id')

In [16]:
pscenes.head()

,image_id,label_eng,label_ja,label_ka,perf_id,play_id,prod_id,pscene_id,scene_id,scene_order,shamisen_id,spuck_id
0,[nan],The Tea Stall at Torii Pass,Torii tōge chamise,鳥居峠茶店,2,86,1,1,313,408,[nan],[nan]
1,[nan],Ao no Dōmon (Blue Tunnel),Ao no Dōmon,青の洞門,2,86,1,2,612,409,[nan],[nan]
2,[nan],The Town of Numazu,Numazu,沼津,3,29,1,3,163,189,[nan],[nan]
3,[nan],House of Heisaku and One Thousand Pine Trees a...,Heisaku uchi yori senbon matsubara,平作内より千本松原,3,29,1,4,641,196,[nan],[nan]
4,[nan],The Maple Viewing Party,Momijigari,紅葉狩,4,47,1,5,287,360,[nan],[nan]


In [17]:
# turn narrators df into join table
narrators = narrators.rename(columns = {'sceneproduction_id':'pscene_id','tayu_id':'narrator_id'})
narrators_pscene = narrators[['narrator_id','pscene_id']]

In [18]:
# drop narrator with id 0 (no such performer exists)
narrators_pscene = narrators_pscene[narrators_pscene['narrator_id'] != '0']

In [19]:
pscenes = pd.merge(pscenes, narrators_pscene, on='pscene_id', how='left')
pscenes = multiValDataFrame(pscenes, 'pscene_id', 'narrator_id')
pscenes.head()

,image_id,label_eng,label_ja,label_ka,narrator_id,perf_id,play_id,prod_id,pscene_id,scene_id,scene_order,shamisen_id,spuck_id
0,[nan],The Tea Stall at Torii Pass,Torii tōge chamise,鳥居峠茶店,[nan],2,86,1,1,313,408,[nan],[nan]
1,[nan],Ao no Dōmon (Blue Tunnel),Ao no Dōmon,青の洞門,[nan],2,86,1,2,612,409,[nan],[nan]
2,[nan],The Town of Numazu,Numazu,沼津,[nan],3,29,1,3,163,189,[nan],[nan]
3,[nan],House of Heisaku and One Thousand Pine Trees a...,Heisaku uchi yori senbon matsubara,平作内より千本松原,[nan],3,29,1,4,641,196,[nan],[nan]
4,[nan],The Maple Viewing Party,Momijigari,紅葉狩,[nan],4,47,1,5,287,360,[nan],[nan]


In [20]:
musicians.head()

,id,sceneproduction_id,kotokokyu_id,kotokokyusub_id,kotokokyu_description,sortorder
0,1,14,116,0,NaN,1
1,2,41,114,0,Yagumo,1
2,3,64,116,0,Koto,1
3,4,153,32,0,Koto,1
4,5,157,114,0,Sankyoku,1


In [21]:
# turn musicians df into join table
musicians = musicians.rename(columns = {'sceneproduction_id':'pscene_id','kotokokyu_id':'musician_id'})
musicians_pscene = musicians[['musician_id','pscene_id']]
musicians_pscene.head()

,musician_id,pscene_id
0,116,14
1,114,41
2,116,64
3,32,153
4,114,157


In [22]:
pscenes = pd.merge(pscenes, musicians_pscene, on='pscene_id', how='left')
pscenes = multiValDataFrame(pscenes, 'pscene_id', 'musician_id')
pscenes.head()

,image_id,label_eng,label_ja,label_ka,musician_id,narrator_id,perf_id,play_id,prod_id,pscene_id,scene_id,scene_order,shamisen_id,spuck_id
0,[nan],The Tea Stall at Torii Pass,Torii tōge chamise,鳥居峠茶店,[nan],[nan],2,86,1,1,313,408,[nan],[nan]
1,[nan],Ao no Dōmon (Blue Tunnel),Ao no Dōmon,青の洞門,[nan],[nan],2,86,1,2,612,409,[nan],[nan]
2,[nan],The Town of Numazu,Numazu,沼津,[nan],[nan],3,29,1,3,163,189,[nan],[nan]
3,[nan],House of Heisaku and One Thousand Pine Trees a...,Heisaku uchi yori senbon matsubara,平作内より千本松原,[nan],[nan],3,29,1,4,641,196,[nan],[nan]
4,[nan],The Maple Viewing Party,Momijigari,紅葉狩,[nan],[nan],4,47,1,5,287,360,[nan],[nan]


In [23]:
pscenes = pscenes.rename(columns = {'pscene_id':'id'})
pscenes = pscenes[['id','label_eng','label_ja','label_ka','play_id','prod_id','perf_id','scene_id','scene_order','narrator_id','musician_id','shamisen_id','spuck_id','image_id']]
pscenes.head()

,id,label_eng,label_ja,label_ka,play_id,prod_id,perf_id,scene_id,scene_order,narrator_id,musician_id,shamisen_id,spuck_id,image_id
0,1,The Tea Stall at Torii Pass,Torii tōge chamise,鳥居峠茶店,86,1,2,313,408,[nan],[nan],[nan],[nan],[nan]
1,2,Ao no Dōmon (Blue Tunnel),Ao no Dōmon,青の洞門,86,1,2,612,409,[nan],[nan],[nan],[nan],[nan]
2,3,The Town of Numazu,Numazu,沼津,29,1,3,163,189,[nan],[nan],[nan],[nan],[nan]
3,4,House of Heisaku and One Thousand Pine Trees a...,Heisaku uchi yori senbon matsubara,平作内より千本松原,29,1,3,641,196,[nan],[nan],[nan],[nan],[nan]
4,5,The Maple Viewing Party,Momijigari,紅葉狩,47,1,4,287,360,[nan],[nan],[nan],[nan],[nan]


In [24]:
# pscenes.to_csv('pscenes_exp.csv', encoding='utf8', index=False)
# pscenes.to_json('pscenes_exp.json', orient='records', force_ascii=False)

In [25]:
# make csv join table of plays + (abstract, performance-agnostic) scenes to add to plays
# play_scenes = pscenes[['play_id','scene_id']]

In [26]:
# play_scenes.to_csv('play_scenes.csv', encoding='utf8', index=False)
# play_scenes.to_json('play_scenes.json', orient='records', force_ascii=False)